In [35]:
import os
import time
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import tiktoken
from tqdm.notebook import tqdm  # 노트북 환경에서 진행률 표시

In [36]:
    # Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("maywell/korean_textbooks",data_files="mmlu_high_school_statistics/train-00000-of-00001.parquet")
ds_train = ds['train']

In [45]:
class CustomDataset(Dataset):
    """
    Hugging Face 데이터셋을 스트리밍 방식으로 처리하는 PyTorch Dataset 클래스.
    """
    def __init__(self, dataset, max_length=1024, stride=512, column_name='0', tokenizer="gpt-4o"):
        
        self.tokenizer = tiktoken.encoding_for_model(tokenizer)         # tiktoken 토크나이저 초기화
        self.dataset = dataset                                          # Hugging Face 데이터셋 저장
        self.max_length = max_length                                    # 최대 시퀀스 길이 저장
        self.stride = stride                                            # 슬라이딩 윈도우의 보폭(stride) 저장
        self.column_name = column_name                                  # 텍스트 데이터가 포함된 컬럼 이름 저장
        
        # 캐시 설정
        self.cache = {}                                                 # 간단한 캐시 딕셔너리
        self.cache_size = 100                                           # 8GB VRAM, 16GB RAM 환경에 맞게 설정
        self.doc_count = len(dataset)                                   # 전체 문서 수 및 인덱스 맵핑
    
    def __len__(self):
        # 첫 번째 문서를 기반으로 평균 청크 수 추정(Loader의 batch추정 및 진행률 위해서 __len__ 필요)
        if self.doc_count > 0:
            # 샘플 문서 토큰화
            sample_doc = self.dataset[0][self.column_name]
            sample_tokens = self.tokenizer.encode(sample_doc)
            
            # 가능한 청크 수 계산
            sample_chunks = max(0, (len(sample_tokens) - self.max_length) // self.stride + 1)
            
            # 전체 청크 수 추정
            return max(self.doc_count, sample_chunks * self.doc_count)
        
        return self.doc_count  # 기본값으로 문서 수 반환
    
    def _process_document(self, doc_idx):
        """
        문서를 처리하여 입력 및 타겟 청크 목록을 반환합니다.
        # token_ids = [15293, 18392, 29843, 38291, 12893, 59302, 71839, 92004]
        # max_length = 5일 때 아래와 같이 첫번째로 데이터를 처리함
        # input_chunk = [15293, 18392, 29843, 38291, 12893]
        # target_chunk = [18392, 29843, 38291, 12893, 59302]
        
        Stride가 2일 때 아래와 같이 두번째로 데이터를 처리함
        # input_chunk = [29843, 38291, 12893, 59302, 71839]
        # target_chunk = [38291, 12893, 59302, 71839, 92004]
        
        # input_chunk와 target_chunk 쌍을 비교해보면 target_chunk는 input_chunk보다 한 토큰 앞으로 이동한 것임을 알 수 있습니다.
        # 즉, max_length만큼 데이터가 증분됨.
        input[0] = 15293   -->   target[0] = 18392 (= input[1])
        input[1] = 18392   -->   target[1] = 29843 (= input[2])
        input[2] = 29843   -->   target[2] = 38291 (= input[3])
        ...
        """
        # 문서 텍스트 가져오기
        doc_text = self.dataset[doc_idx][self.column_name]
        
        # tiktoken으로 토큰화
        token_ids = self.tokenizer.encode(doc_text)
        
        input_chunks = []
        target_chunks = []
        
        # 슬라이딩 윈도우를 사용하여 토큰 ID 리스트를 청크로 나눔
        for i in range(0, len(token_ids) - self.max_length, self.stride):
            # 입력 청크: 인덱스 i부터 시작하는 max_length 길이의 토큰 시퀀스
            input_chunk = token_ids[i : i + self.max_length]
            
            # 타겟 청크: 입력보다 한 토큰 뒤에서 시작 (자동회귀 예측을 위한 1토큰 증분)
            # 예: 입력=[1,2,3,4], 타겟=[2,3,4,5]
            target_chunk = token_ids[i + 1 : i + self.max_length + 1]
            
            # 입력 청크와 타겟 청크의 길이가 모두 max_length인지 확인
            if len(input_chunk) == self.max_length and len(target_chunk) == self.max_length:
                # 텐서로 변환하여 리스트에 추가
                input_chunks.append(torch.tensor(input_chunk, dtype=torch.long))
                target_chunks.append(torch.tensor(target_chunk, dtype=torch.long))
        
        return input_chunks, target_chunks
    
    def __getitem__(self, idx):
        """
        인덱스에 해당하는 데이터 항목을 반환합니다.
        스트리밍 방식으로 필요할 때 문서를 처리합니다.
        """
        # 캐시 확인
        # 이미 처리한 인덱스는 캐시에 저장해두고, 다시 요청이 들어오면 재계산 없이 바로 반환합니다.
        if idx in self.cache:
            return self.cache[idx]
        
        # 문서 인덱스 계산(입력된 idx를 문서 총 개수(self.doc_count)로 나눈 나머지를 사용합니다.)
        # 예시: idx=15, 문서 수가 10개라면, doc_idx=5가 됩니다.
        # 이렇게 하면 idx가 문서 수보다 커져도 항상 유효한 문서 인덱스를 얻을 수 있습니다.
        doc_idx = idx % self.doc_count
        
        """
        [참고]
        유한한 문서 집합에서 무한한 인덱스 처리:

        데이터셋에는 유한한 수의 문서(예: 1,000개)가 있습니다.
        하지만 __getitem__은 이론적으로 매우 큰 인덱스(예: 5,000)를 요청받을 수 있습니다.
        이때 idx % self.doc_count 연산으로 실제 사용 가능한 문서 인덱스로 변환합니다.


        순환 접근 패턴:

        이 방식은 문서들을 계속해서 순환하며 접근할 수 있게 합니다.
        예: 문서가 3개(0, 1, 2)일 때, idx가 3이면 다시 문서 0으로, 4면 문서 1로 돌아갑니다.


        예시로 설명:

        문서가 5개 있다고 가정합시다(인덱스 0~4).
        idx = 7이 들어오면, 7 % 5 = 2가 되어 세 번째 문서(인덱스 2)에 접근합니다.
        idx = 12가 들어오면, 12 % 5 = 2가 되어 동일하게 세 번째 문서에 접근합니다.
        """

        
        
        # 문서 처리(고정슬라이드드)

        input_chunks, target_chunks = self._process_document(doc_idx)
        
        # 처리된 청크가 없으면 다음 문서 시도
        if not input_chunks:
            return self.__getitem__((idx + 1) % len(self))
        
        # 청크 인덱스 계산
        # 청크로 문서를 나누기 때문에 마찬가지로 필요
        
        chunk_idx = (idx // self.doc_count) % max(1, len(input_chunks))
        """
        [참고]
        문서가 3개 있고, 각 문서에서 생성된 청크 수가 다음과 같다고 가정해봅시다:

        문서 0: 4개 청크
        문서 1: 2개 청크
        문서 2: 3개 청크

        chunk사이즈가 5라면, idx가 0~4일 때는 문서 0의 청크를 가져오고,
        idx가 5~9일 때는 문서 1의 청크를 가져오고, idx가 10~14일 때는 문서 2의 청크를 가져옵니다. 
        3번이 순환이 되면서 15번째 idx를 요구한다면면 자동으로 문서의 크기로 나눠 주기 때문에 index오류를 방지할 수 있습니다.
        """



        # 결과 저장
        result = (input_chunks[chunk_idx % len(input_chunks)], target_chunks[chunk_idx % len(input_chunks)])
        
        # 캐시 업데이트 (제한된 크기)
        if len(self.cache) < self.cache_size:
            self.cache[idx] = result
        
        return result

In [46]:
print("=== 데이터셋 정보 ===")
print(f"데이터셋 크기: {len(ds_train)} 개의 문서")
print(f"데이터셋 컬럼: {ds_train.column_names}")
print("\n=== 첫 번째 문서 샘플 ===")
first_doc = ds_train[0]
column_name = '0'
print(f"문서 내용 (처음 300자): {first_doc[column_name][:300]}...")

=== 데이터셋 정보 ===
데이터셋 크기: 76033 개의 문서
데이터셋 컬럼: ['0']

=== 첫 번째 문서 샘플 ===
문서 내용 (처음 300자): "도서관 vs. 책 가게, 어느 것이 좀 더 중요한가?"에 대한 토론 내용:

**Phi:** 안녕, Epsilon. 오늘은 도서관과 서점에 대해 이야기하고 싶다.

**Epsilon:** 네, Phi. 무슨 이야기인지 궁금하다.

**Phi:** 도서관과 책 가게는 모두 책을 제공한다는 점에서 비슷하다. 하지만 두 가지 장소는 서로 다른 점도 많다. 도서관은 공공 기관이며, 책을 무료로 대출할 수 있다. 반면에 서점은 사설 기업이며, 책을 판매한다.

**Epsilon:** 네, 그렇다. 그리고 도서관은 일반적으로 책의 종류가 더...


In [47]:
tokenizer = tiktoken.encoding_for_model("gpt-4o")
sample_text = first_doc[column_name][:100]  # 문서의 처음 100자
tokens = tokenizer.encode(sample_text)
decoded_tokens = [tokenizer.decode([token]) for token in tokens]
print(f"원문: {sample_text}")
print(f"토큰 개수: {len(tokens)}")
print(f"토큰 ID: {tokens[:20]}...")
print(f"디코딩된 토큰: {decoded_tokens[:20]}...")

원문: "도서관 vs. 책 가게, 어느 것이 좀 더 중요한가?"에 대한 토론 내용:

**Phi:** 안녕, Epsilon. 오늘은 도서관과 서점에 대해 이야기하고 싶다.

**Epsil
토큰 개수: 51
토큰 ID: [1, 5827, 4865, 15853, 10217, 13, 67671, 9919, 7996, 11, 138744, 46947, 107894, 28526, 141410, 4081, 16842, 3107, 33398, 68258]...
디코딩된 토큰: ['"', '도', '서', '관', ' vs', '.', ' 책', ' 가', '게', ',', ' 어느', ' 것이', ' 좀', ' 더', ' 중요한', '가', '?"', '에', ' 대한', ' 토']...


In [51]:
# 3. CustomDataset 객체 생성 및 테스트
# 짧은 max_length와 stride 값을 사용하여 예제 결과 확인
max_length = 10  # 시각화를 위해 작은 값 사용
stride = 5
custom_ds = CustomDataset(ds_train, max_length=max_length, stride=stride, column_name=column_name,tokenizer="gpt-4o")
print(f"\n=== CustomDataset 정보 ===")
print(f"총 추정 청크 수: {len(custom_ds)}")


=== CustomDataset 정보 ===
총 추정 청크 수: 9884290


In [ ]:
import pandas as pd 
# 4. 첫 번째 문서를 토큰화하고 슬라이딩 윈도우 시각화
first_doc_text = ds_train[0][column_name]
first_doc_tokens = tokenizer.encode(first_doc_text)
    
print(f"\n=== 슬라이딩 윈도우 처리 ===")
print(f"첫 번째 문서의 토큰 수: {len(first_doc_tokens)}")
window_results = []
for i in range(0, len(first_doc_tokens) - max_length, stride):
    input_chunk = first_doc_tokens[i : i + max_length]
    print
    target_chunk = first_doc_tokens[i + 1 : i + max_length + 1]
    
    if len(input_chunk) == max_length and len(target_chunk) == max_length:
        input_decoded = tokenizer.decode(input_chunk)
        target_decoded = tokenizer.decode(target_chunk)
        window_results.append({
            "시작 인덱스": i,
            "입력 청크 ID": input_chunk,
            "입력 청크 텍스트": input_decoded,
            "타겟 청크 ID": target_chunk,
            "타겟 청크 텍스트": target_decoded
        })
#  
results_df = pd.DataFrame(window_results)
results_df.head()



=== 슬라이딩 윈도우 처리 ===
첫 번째 문서의 토큰 수: 659


,시작 인덱스,입력 청크 ID,입력 청크 텍스트,타겟 청크 ID,타겟 청크 텍스트
0,0,"[1, 5827, 4865, 15853, 102...","""도서관 vs. 책 가게,","[5827, 4865, 15853, 10217,...","도서관 vs. 책 가게, 어느"
1,5,"[13, 67671, 9919, 7996, 11...",". 책 가게, 어느 것이 좀 더 중요한","[67671, 9919, 7996, 11, 13...","책 가게, 어느 것이 좀 더 중요한가"
2,10,"[138744, 46947, 107894, 28...","어느 것이 좀 더 중요한가?""에 대한 토","[46947, 107894, 28526, 141...","것이 좀 더 중요한가?""에 대한 토론"
3,15,"[4081, 16842, 3107, 33398,...","가?""에 대한 토론 내용:\n\n**Phi","[16842, 3107, 33398, 68258...","?""에 대한 토론 내용:\n\n**Phi:**"
4,20,"[39168, 69634, 1402, 410, ...","론 내용:\n\n**Phi:** 안녕, E","[69634, 1402, 410, 72002, ...","내용:\n\n**Phi:** 안녕, Epsilon"


In [56]:

# 실제 처리 결과 확인을 위한 코드
def visualize_dataset_processing():
    # 1. 데이터셋 기본 정보 출력
    print("=== 데이터셋 정보 ===")
    print(f"데이터셋 크기: {len(ds_train)} 개의 문서")
    print(f"데이터셋 컬럼: {ds_train.column_names}")
    
    # 첫 번째 문서 출력
    print("\n=== 첫 번째 문서 샘플 ===")
    first_doc = ds_train[0]
    column_name = '0' if '0' in ds_train.column_names else ds_train.column_names[0]
    print(f"문서 내용 (처음 300자): {first_doc[column_name][:300]}...")
    
    # 2. 토큰화 과정 시각화
    print("\n=== 토큰화 과정 ===")
    sample_text = first_doc[column_name][:100]  # 문서의 처음 100자
    tokens = tokenizer.encode(sample_text)
    decoded_tokens = [tokenizer.decode([token]) for token in tokens]
    
    print(f"원문: {sample_text}")
    print(f"토큰 개수: {len(tokens)}")
    print(f"토큰 ID: {tokens[:20]}...")
    print(f"디코딩된 토큰: {decoded_tokens[:20]}...")
    
    # 3. CustomDataset 객체 생성 및 테스트
    # 짧은 max_length와 stride 값을 사용하여 예제 결과 확인
    max_length = 10  # 시각화를 위해 작은 값 사용
    stride = 5
    custom_ds = CustomDataset(ds_train, max_length=max_length, stride=stride, column_name=column_name)
    
    print(f"\n=== CustomDataset 정보 ===")
    print(f"총 추정 청크 수: {len(custom_ds)}")
    
    # 4. 첫 번째 문서를 토큰화하고 슬라이딩 윈도우 시각화
    first_doc_text = ds_train[0][column_name]
    first_doc_tokens = tokenizer.encode(first_doc_text)
    
    print(f"\n=== 슬라이딩 윈도우 처리 ===")
    print(f"첫 번째 문서의 토큰 수: {len(first_doc_tokens)}")
    
    # 처리 결과를 DataFrame으로 시각화
    window_results = []
    for i in range(0, len(first_doc_tokens) - max_length, stride):
        input_chunk = first_doc_tokens[i : i + max_length]
        target_chunk = first_doc_tokens[i + 1 : i + max_length + 1]
        
        if len(input_chunk) == max_length and len(target_chunk) == max_length:
            input_decoded = tokenizer.decode(input_chunk)
            target_decoded = tokenizer.decode(target_chunk)
            window_results.append({
                "시작 인덱스": i,
                "입력 청크 ID": input_chunk,
                "입력 청크 텍스트": input_decoded,
                "타겟 청크 ID": target_chunk,
                "타겟 청크 텍스트": target_decoded
            })
    
    if window_results:
        results_df = pd.DataFrame(window_results)
        pd.set_option('display.max_colwidth', 30)  # 셀 내용 표시 길이 제한
        print(results_df)
    else:
        print("슬라이딩 윈도우로 생성된 청크가 없습니다. 문서가 너무 짧거나 max_length가 너무 큽니다.")
    
    # 5. __getitem__ 메서드 테스트
    print("\n=== __getitem__ 메서드 테스트 ===")
    for idx in range(3):  # 처음 3개 항목 테스트
        try:
            input_tensor, target_tensor = custom_ds[idx]
            print(f"\n항목 인덱스 {idx}:")
            print(f"문서 인덱스: {idx % custom_ds.doc_count}")
            print(f"입력 텐서 모양: {input_tensor.shape}")
            print(f"타겟 텐서 모양: {target_tensor.shape}")
            print(f"입력 텐서 (디코딩): {tokenizer.decode(input_tensor.tolist())}")
            print(f"타겟 텐서 (디코딩): {tokenizer.decode(target_tensor.tolist())}")
            
            # 시각화: 입력 및 타겟 비교
            input_decoded = [tokenizer.decode([t]) for t in input_tensor.tolist()]
            target_decoded = [tokenizer.decode([t]) for t in target_tensor.tolist()]
            
            print("\n입력-타겟 토큰 페어 비교:")
            for i, (inp, tar) in enumerate(zip(input_decoded, target_decoded)):
                print(f"인덱스 {i}: 입력='{inp}' -> 타겟='{tar}'")
        except Exception as e:
            print(f"인덱스 {idx}에서 오류 발생: {e}")
    
    # 6. 슬라이딩 윈도우 시각화
    if len(first_doc_tokens) > max_length:
        plt.figure(figsize=(12, 6))
        tokens_to_show = min(30, len(first_doc_tokens))  # 처음 30개 토큰만 표시
        
        # 토큰값 표시
        plt.subplot(211)
        plt.bar(range(tokens_to_show), first_doc_tokens[:tokens_to_show], color='skyblue')
        plt.xlabel('토큰 인덱스')
        plt.ylabel('토큰 ID')
        plt.title('문서의 처음 몇 개 토큰')
        
        # 슬라이딩 윈도우 시각화
        plt.subplot(212)
        
        windows = []
        for i in range(0, min(tokens_to_show, len(first_doc_tokens) - max_length), stride):
            window = np.zeros(tokens_to_show)
            window[i:i+max_length] = 1
            windows.append(window)
        
        if windows:
            windows_array = np.array(windows)
            plt.imshow(windows_array, aspect='auto', cmap='Greens')
            plt.xlabel('토큰 인덱스')
            plt.ylabel('윈도우 번호')
            plt.title('슬라이딩 윈도우 시각화')
        else:
            plt.text(0.5, 0.5, '슬라이딩 윈도우를 생성할 수 없습니다.', 
                     horizontalalignment='center', verticalalignment='center')
        
        plt.tight_layout()
        plt.show()

# 처리 결과 시각화 실행
visualize_dataset_processing()

=== 데이터셋 정보 ===
데이터셋 크기: 76033 개의 문서
데이터셋 컬럼: ['0']

=== 첫 번째 문서 샘플 ===
문서 내용 (처음 300자): "도서관 vs. 책 가게, 어느 것이 좀 더 중요한가?"에 대한 토론 내용:

**Phi:** 안녕, Epsilon. 오늘은 도서관과 서점에 대해 이야기하고 싶다.

**Epsilon:** 네, Phi. 무슨 이야기인지 궁금하다.

**Phi:** 도서관과 책 가게는 모두 책을 제공한다는 점에서 비슷하다. 하지만 두 가지 장소는 서로 다른 점도 많다. 도서관은 공공 기관이며, 책을 무료로 대출할 수 있다. 반면에 서점은 사설 기업이며, 책을 판매한다.

**Epsilon:** 네, 그렇다. 그리고 도서관은 일반적으로 책의 종류가 더...

=== 토큰화 과정 ===
원문: "도서관 vs. 책 가게, 어느 것이 좀 더 중요한가?"에 대한 토론 내용:

**Phi:** 안녕, Epsilon. 오늘은 도서관과 서점에 대해 이야기하고 싶다.

**Epsil
토큰 개수: 51
토큰 ID: [1, 5827, 4865, 15853, 10217, 13, 67671, 9919, 7996, 11, 138744, 46947, 107894, 28526, 141410, 4081, 16842, 3107, 33398, 68258]...
디코딩된 토큰: ['"', '도', '서', '관', ' vs', '.', ' 책', ' 가', '게', ',', ' 어느', ' 것이', ' 좀', ' 더', ' 중요한', '가', '?"', '에', ' 대한', ' 토']...

=== CustomDataset 정보 ===
총 추정 청크 수: 9884290

=== 슬라이딩 윈도우 처리 ===
첫 번째 문서의 토큰 수: 659
     시작 인덱스                       입력 청크 ID                입력 청크 텍스트  \
0         0  [1, 5827, 4865, 15853, 102...          

NameError: name 'plt' is not defined

In [ ]:
# 데이터 로딩 및 데이터로더 생성 함수
def create_dataloader(batch_size=4, max_length=1024, stride=512, shuffle=True, num_workers=2, column_name='0'):
    """
    스트리밍 방식으로 데이터를 처리하는 데이터로더를 생성합니다.
    """
    print("데이터셋 로딩 중...")
    
    # 데이터셋 로드
    ds = load_dataset("maywell/korean_textbooks", data_files="mmlu_high_school_statistics/train-00000-of-00001.parquet")
    ds_train = ds['train']
    
    print(f"데이터셋 로드 완료: {len(ds_train)} 문서")
    
    # CustomDataset 생성
    print("데이터셋 객체 생성 중...")
    dataset = CustomDataset(
        ds_train,
        max_length=max_length,
        stride=stride,
        column_name=column_name
    )
    
    # DataLoader 생성
    print("데이터로더 생성 중...")
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers if torch.cuda.is_available() else 0,  # CPU 전용 모드에서는 0으로 설정
        pin_memory=torch.cuda.is_available()  # GPU 사용 시 메모리 고정
    )
    
    print(f"데이터로더 생성 완료: batch_size={batch_size}, max_length={max_length}")
    return dataloader, dataset.tokenizer


# 데이터로더 테스트 및 시각화 함수
def test_dataloader(dataloader, num_batches=3):
    """
    데이터로더를 테스트하고 첫 몇 개 배치의 정보를 출력합니다.
    """
    print("데이터로더 테스트 중...")
    tokenizer = tiktoken.encoding_for_model("gpt-4o")
    
    for i, (input_ids, target_ids) in enumerate(tqdm(dataloader, total=num_batches)):
        if i >= num_batches:
            break
        
        print(f"배치 {i+1}: 입력 형태: {input_ids.shape}, 타겟 형태: {target_ids.shape}")
        
        # 첫 번째 배치의 샘플 토큰 확인
        if i == 0:
            # 첫 번째 시퀀스의 처음 10개 토큰
            first_tokens = input_ids[0][:10].tolist()
            first_target_tokens = target_ids[0][:10].tolist()
            
            print(f"샘플 입력 토큰: {first_tokens}")
            print(f"샘플 타겟 토큰: {first_target_tokens}")
            
            # 증분 관계 확인 (자동회귀 패턴)
            print("\n자동회귀 패턴 확인:")
            for j in range(min(5, len(first_tokens))):
                if j+1 < len(first_tokens):
                    print(f"입력[{j+1}] = {input_ids[0][j+1]} == 타겟[{j}] = {target_ids[0][j]}: {input_ids[0][j+1] == target_ids[0][j]}")
            
            # 샘플 텍스트 디코딩
            try:
                sample_text = tokenizer.decode(input_ids[0][:30].tolist())
                print(f"\n샘플 텍스트 (처음 30개 토큰): {sample_text}")
            except Exception as e:
                print(f"텍스트 디코딩 실패: {e}")


# 메인 실행 코드 - 노트북 환경에서 바로 실행 가능
if __name__ == "__main__":
    # 이 코드는 Jupyter Notebook에서 직접 실행하거나
    # 아래와 같이 셀에서 따로 불러올 수 있습니다.
    
    # CUDA 사용 가능 여부 확인
    print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    
    # 하드웨어에 맞게 설정 조정
    batch_size = 4        # 8GB VRAM에 맞게 조정
    max_length = 1024     # 8GB VRAM에 맞게 조정
    stride = 512          # 절반 오버랩
    num_workers = 2       # 16GB RAM에 맞게 조정
    
    # 데이터로더 생성
    loader, tokenizer = create_dataloader(
        batch_size=batch_size,
        max_length=max_length,
        stride=stride,
        num_workers=num_workers
    )
    
    # 데이터로더 테스트
    test_dataloader(loader)

In [ ]:
# 데이터 시각화 함수 - 배치 내 토큰 분포 확인
def visualize_batch_tokens(dataloader, batch_idx=0):
    """
    배치 내 토큰 분포를 시각화합니다.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    from collections import Counter
    
    # 배치 가져오기
    for i, (input_ids, target_ids) in enumerate(dataloader):
        if i == batch_idx:
            batch = input_ids
            break
    
    # 배치를 numpy 배열로 변환
    batch_np = batch.numpy()
    
    # 모든 토큰을 하나의 리스트로 평탄화
    all_tokens = batch_np.flatten()
    
    # 토큰 빈도 계산
    token_counts = Counter(all_tokens)
    most_common = token_counts.most_common(20)
    
    # 시각화
    plt.figure(figsize=(12, 6))
    
    # 상위 20개 토큰 빈도
    tokens, counts = zip(*most_common)
    plt.subplot(1, 2, 1)
    plt.bar(range(len(tokens)), counts)
    plt.xticks(range(len(tokens)), [str(t) for t in tokens], rotation=45)
    plt.title('상위 20개 토큰 빈도')
    plt.xlabel('토큰 ID')
    plt.ylabel('빈도')
    
    # 토큰 히스토그램
    plt.subplot(1, 2, 2)
    plt.hist(all_tokens, bins=50)
    plt.title('토큰 분포 히스토그램')
    plt.xlabel('토큰 ID')
    plt.ylabel('빈도')
    
    plt.tight_layout()
    plt.show()
    
    # 토큰 디코딩 시도
    tokenizer = tiktoken.encoding_for_model("gpt-4o")
    for token, count in most_common[:10]:
        try:
            decoded = tokenizer.decode([int(token)])
            print(f"토큰 ID {token}: '{decoded}' (빈도: {count})")
        except:
            print(f"토큰 ID {token}: 디코딩 불가 (빈도: {count})")


# 자동회귀 패턴 시각적 확인
def check_autoregressive_pattern(dataloader):
    """
    데이터로더의 자동회귀 패턴을 확인합니다.
    """
    import matplotlib.pyplot as plt
    
    # 첫 번째 배치 가져오기
    for input_ids, target_ids in dataloader:
        break
    
    # 첫 번째 샘플의 처음 20개 토큰
    input_sample = input_ids[0][:20].numpy()
    target_sample = target_ids[0][:20].numpy()
    
    # 시각화
    plt.figure(figsize=(12, 6))
    plt.plot(input_sample, 'b-', label='입력 토큰')
    plt.plot(target_sample, 'r-', label='타겟 토큰')
    plt.legend()
    plt.title('입력과 타겟의 자동회귀 패턴')
    plt.xlabel('위치')
    plt.ylabel('토큰 ID')
    plt.grid(True)
    
    # 오프셋 확인
    offset_check = input_sample[1:] == target_sample[:-1]
    all_match = offset_check.all()
    
    plt.figtext(0.5, 0.01, f"자동회귀 패턴 일치: {all_match}", 
                ha="center", fontsize=12, 
                bbox={"facecolor":"orange" if not all_match else "green", 
                      "alpha":0.5, "pad":5})
    
    plt.tight_layout()
    plt.show()
    
    # 상세 비교
    print("\n입력과 타겟 토큰 비교:")
    for i in range(len(input_sample)-1):
        match = input_sample[i+1] == target_sample[i]
        print(f"입력[{i+1}]={input_sample[i+1]} vs 타겟[{i}]={target_sample[i]}: {'✓' if match else '✗'}")